In [1]:
import os
import time
import pickle
import numpy as np
import pandas as pd
from model import Schelling
from SALib.sample import saltelli

exp_path = 'gsa-schelling-radius-05-10-2022'

# Create folder structure
if not os.path.exists(exp_path):
    os.makedirs(exp_path)
    
if not os.path.exists(exp_path + '/data'):
    os.makedirs(exp_path + '/data')

print(os.cpu_count())

40


In [2]:
n_samples = 2048
calc_sec_order = True
var_params = {
                'density':{'bounds':[0.5, 0.95],'label':'Density'},
                'minority_pc':{'bounds':[0.5, 0.95],'label':'%Blue'},
                'mu1':{'bounds':[0, 1],'label':'Mean homophily (blue)'},
                'std1':{'bounds':[0, 0.2],'label':'Std. homophily (blue)'},
                'mu2':{'bounds':[0, 1],'label':'Mean homophily (orange)'},
                'std2':{'bounds':[0, 0.2],'label':'Std. homophily (orange)'},
                'radius':{'bounds':[0, 5], 'label':'Radius'}
            }
names = list(var_params.keys())
bounds = [par['bounds'] for par in var_params.values()]

problem = {'num_vars': len(names), 'names': names, 'bounds': bounds}
par_values = saltelli.sample(problem, n_samples, calc_sec_order)

# Fixed parameters
par_frame = pd.DataFrame(columns=names, data=par_values)
par_frame['radius'] = par_frame['radius'].apply(np.ceil).astype(int)
fixed_params = {'height':100, 'width':100, 'move_fraction':0.15,
                'mode':'Heterogeneous', 'std':0.02,
                'homophily':0.3, 'torus':True, 'max_steps':300,
                'window_size':30, 'conv_threshold':0.01}
par_frame['filename'] = [exp_path + '/data/file' + str(i) for i in range(par_values.shape[0])]

# Add the fixed parameters to the frame
for fixed_par, value in fixed_params.items():
    par_frame[fixed_par] = value

dask_list = list(par_frame.T.to_dict().values())
print(len(dask_list), dask_list[0])

32768 {'density': 0.50032958984375, 'minority_pc': 0.64117431640625, 'mu1': 0.734130859375, 'std1': 0.14936523437500002, 'mu2': 0.319580078125, 'std2': 0.147119140625, 'radius': 5, 'filename': 'gsa-schelling-radius-05-10-2022/data/file0', 'height': 100, 'width': 100, 'move_fraction': 0.15, 'mode': 'Heterogeneous', 'std': 0.02, 'homophily': 0.3, 'torus': True, 'max_steps': 300, 'window_size': 30, 'conv_threshold': 0.01}


In [3]:
import dask
from dask.diagnostics import ProgressBar

@dask.delayed
def simulator(pars):
    """Perform a single model run."""
    model = Schelling(pars=pars)
    model.simulate(export=False)
    return (model.avg_fraction_sim, model.fraction_sat_1, 
            model.fraction_sat_2, model.convergence_metric[-1])

results = []
for pars in dask_list:
    model_run = simulator(pars)
    results.append(model_run)

In [4]:
pbar = ProgressBar()
pbar.register()

In [5]:
final_res = dask.compute(results, scheduler='processes', num_workers=40)

[###                                     ] | 9% Completed | 12m 53sss

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[####################################    ] | 90% Completed | 113m 46s

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[######################################  ] | 97% Completed | 2hr 2m9s

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
pbar.unregister()

In [7]:
par_frame[['avg_fraction_sim', 'fraction_sat_1', 'fraction_sat_2', 'fraction_sat']] = final_res[0]

In [8]:
par_frame.to_pickle(exp_path + '/data/frame.pkl')
with open(f'{exp_path}/problem.pickle', 'wb') as handle:
    pickle.dump(problem, handle, protocol=4)